In [ ]:
!pip install torch==2.0.0+cu118 torchvision==0.15.1+cu118 torchaudio==2.0.1 --index-url https://download.pytorch.org/whl/cu118
!pip install pyg_lib torch_scatter torch_sparse torch_cluster torch_spline_conv -f https://data.pyg.org/whl/torch-2.0.0+cu118.html
!pip install torch-geometric

import torch
print(torch.__version__)



In [ ]:
import torch
import torch_geometric
import pyg_lib
import torch_sparse
from torch_geometric.sampler import NegativeSampling, NeighborSampler
print(f"Torch version: {torch.__version__}")
print(f"Cuda available: {torch.cuda.is_available()}")
print(pyg_lib.__version__)
print(torch_sparse.__version__)
print(f"Torch geometric version: {torch_geometric.__version__}")
import torch.nn.functional as F
from torch_geometric.data import Dataset, Data
import torch_geometric as pyg
from torch_geometric.loader import LinkNeighborLoader
from torch_geometric.nn import SAGEConv, to_hetero
import torch_geometric.transforms as T
from torch_geometric.data import HeteroData
from torch_geometric.transforms import RandomLinkSplit
from torch_geometric.utils import negative_sampling
from torch import nn
from torch import Tensor
from tqdm import tqdm

In [ ]:


class GNN(torch.nn.Module):
    def __init__(self, hidden_channels):
        super().__init__()
        self.conv1 = SAGEConv(hidden_channels, hidden_channels)
        self.conv2 = SAGEConv(hidden_channels, hidden_channels)

    def forward(self, x: Tensor, edge_index: Tensor) -> Tensor:
        # print("masuk")
        x = F.relu(self.conv1(x, edge_index))
        # print("masuk1")
        x = self.conv2(x, edge_index)
        # print("masuk2")
        return x


class Classifier(torch.nn.Module):
    def forward(self, x_user: Tensor, x_movie: Tensor, edge_label_index: Tensor) -> Tensor:

        edge_feat_user = x_user[edge_label_index[0]]
        edge_feat_movie = x_movie[edge_label_index[1]]
        # print("Shape")
        # print(edge_feat_user.shape)
        # print(edge_feat_movie.shape)
        # print((edge_feat_user * edge_feat_movie).sum(dim=-1).shape)
        return (edge_feat_user * edge_feat_movie).sum(dim=-1)


class Model(torch.nn.Module):
    def __init__(self, hidden_channels):
        super().__init__()
        # Since the dataset does not come with rich features, we also learn two
        # embedding matrices for users and movies:
        self.movie_lin = torch.nn.Linear(89, hidden_channels)
        self.user_lin = torch.nn.Linear(2, hidden_channels)
        self.user_emb = torch.nn.Embedding(
            input_data_test["user"].num_nodes, hidden_channels)
        self.movie_emb = torch.nn.Embedding(
            input_data_test["movie"].num_nodes, hidden_channels)
        # Instantiate homogeneous GNN:
        self.gnn = GNN(hidden_channels)
        # Convert GNN model into a heterogeneous variant:
        self.gnn = to_hetero(self.gnn, metadata=input_data_test.metadata())
        self.classifier = Classifier()

    def forward(self, data: HeteroData) -> Tensor:

        movie_emb_dict = self.movie_emb(input_data_test["movie"].node_id)
        movie_lin_dict = self.movie_lin(input_data_test["movie"].x)

        x_dict = {
            "user": self.user_lin(input_data_test["user"].x) + self.user_emb(input_data_test["user"].node_id),
            # self.user_lin(data["user"].x) + self.user_emb(data["user"].node_id),
            "movie": movie_emb_dict + movie_lin_dict,
        }
        # print("EDGE INDEX DICT")
        # print(data.edge_index_dict)

        x_dict = self.gnn(x_dict, data.edge_index_dict)
        pred = self.classifier(
            x_dict["user"],
            x_dict["movie"],
            data["user", "rating", "movie"].edge_index,
        )
        return pred


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import os
print(os.listdir())
os.chdir("sample_data")

In [ ]:
# print("LOAD!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!")
input_data_test = torch.load('../drive/MyDrive/Anime/input_data_test_user_id_0.pt')
print(input_data_test)

In [ ]:
checkpoint_path = '../drive/MyDrive/Anime/Anime_GNN_updated.pt'
checkpoint = torch.load(checkpoint_path)
# Load the model
model = Model(hidden_channels=64)
model.load_state_dict(checkpoint)

# Move the model to the desired device (e.g., GPU)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

In [ ]:
input_data_test.to(device)

In [ ]:
test_preds = []
testing_data.to(device)
test_preds.append(model(testing_data))


In [ ]:
top_k_values, top_k_indices = torch.topk(test_preds[0], k=10)

# The 'top_k_values' tensor contains the top 10 elements.
# The 'top_k_indices' tensor contains the indices of the top 10 elements.

print("Top 10 Values:")
print(top_k_values)
print("\nTop 10 Indices:")
print(top_k_indices)


In [ ]:
top_10_indices = top_k_indices.tolist()

prediction_list = []
for index in top_10_indices:
  print(testing_data['user', 'rating', 'movie'].edge_index[0][index])
  print(index)
  prediction_list.append(testing_data['user', 'rating', 'movie'].edge_index[1][index])

In [ ]:
prediction_list